## Exemplo de classificador com leitura do CSV. 

Etapas:

- Leitura CSV.
- Tratamento de imagens com lib PIL. 
- Leitura imagem. 
- Aplicacao Filtro para remocao background


Input do modelo : (Imagem, sexo, idade, etnia)

In [ ]:
!pip install -r requirements.txt

In [1]:
import pandas as pd
from PIL import Image, ImageChops
import numpy as np
import torch
from lib.ImageFIlter import treat_image_PIL

In [17]:
ds_file = pd.read_csv('dataset/exemplo_csv_2.csv')


## Geracao Tensor com imagens filtradas

In [ ]:
## Loop
img_dataset = np.ones((ds_file.shape[0],256,256),dtype=np.uint8)

j=0
for i in ds_file['path']:
    img_dataset[j]=treat_image_PIL('dataset/path/'+i)
    j+=1
tensor_imagem = torch.tensor(img_dataset)


## Geracao de rótulos para classificação

Colunas utilizadas para classificacao (Labels)

labels = [ 'FA', 'TA/Flutter',
       'TPSV/TS', 'TV/FV', 'BAV 2º/3º/Avanc./BS', 'Supra ST',
       'Corrente de Lesao', 'Extrassistole', 'BRD', 'BRE', 'MP', 'Normal',
       'Outros', 'Exclusão']

In [23]:
ds_file.columns

Index(['path', 'image_id', 'ID', 'IDADE', 'SEXO', 'ETNIA', 'FA', 'TA/Flutter',
       'TPSV/TS', 'TV/FV', 'BAV 2º/3º/Avanc./BS', 'Supra ST',
       'Corrente de Lesao', 'Extrassistole', 'BRD', 'BRE', 'MP', 'Normal',
       'Outros', 'Exclusão'],
      dtype='object')

In [57]:
##
labels = [ 'FA', 'TA/Flutter',
       'TPSV/TS', 'TV/FV', 'BAV 2º/3º/Avanc./BS', 'Supra ST',
       'Corrente de Lesao', 'Extrassistole', 'BRD', 'BRE', 'MP', 'Normal']

In [ ]:

ds_labels = ds_file[labels]
ds_labels=ds_labels.astype(int)#,'False':0})
tensor_label = torch.tensor(np.array(ds_labels))

## Geracao Tensor Age, sex, Etinia

In [71]:
ds_id_sex_et= ds_file[['IDADE', 'SEXO', 'ETNIA']]

In [72]:
ds_id_sex_et

,IDADE,SEXO,ETNIA
0,85,M,BRANCA
1,32,F,BRANCA
2,60,F,BRANCA
3,57,F,BRANCA
4,57,F,BRANCA
5,57,F,BRANCA
6,57,F,BRANCA
7,57,F,BRANCA
8,57,F,BRANCA
9,57,F,BRANCA
